In [1]:
import os
import torch
import pandas as pd
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Amazon


#### globals
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.DataFrame(columns=['depth', "heads", "epochs", "activation_func", "final_train_score", "valid_score", "test_score"])

## tuning params
hyper_parameters = {
    "depth": list(range(1,6)),
    "heads": list(range(1, 9)),
    "epochs": list(range(50, 550, 50)),
    "activation_func": [F.elu, F.relu, F.tanh]
}

## load datset
dataset = 'Photo'
path = os.path.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())

## seeds
torch.manual_seed(0)

## add train / valid / test masks
dataset.shuffle()
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)

print(dataset.num_classes) # 8
# 8 * 20 --> 160 
data.train_mask[:160] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[160:660] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[660:1660] = True

data.to(device)

8


Data(edge_index=[2, 238162], test_mask=[7650], train_mask=[7650], val_mask=[7650], x=[7650, 745], y=[7650])

In [2]:
from torch_geometric.nn import Sequential, GATConv, GCNConv
from torch.nn import ELU, Dropout, ReLU
from torch.nn import ModuleList

class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, config):
        super(Net, self).__init__()
        
        self.num_layers = config['depth']
        self.act = config['activation_func']
        self.dropout = 0.6
        
        self.convs = ModuleList()
        self.convs.append(
            GATConv(in_channels, 8, heads=config['heads'], dropout=self.dropout)
        )
        
        for i in range(1, self.num_layers):
            layer = None
            if i == self.num_layers - 1:
                # last layer
                layer = GATConv(config['heads'] * 8,  out_channels, heads=1, dropout=self.dropout)
            else:
                layer = GATConv(config['heads'] * 8,  8, heads=config['heads'], dropout=self.dropout)
                
            self.convs.append(layer)
        

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if self.act is not None:
                x = self.act(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            
        return F.log_softmax(x, dim= -1)

In [3]:
def log_stats(param_dict, train_acc, valid_acc, test_acc, filename):
    global df 
    
    df = df.append({
                'depth': param_dict['depth'], 
                "heads": param_dict['heads'], 
                "epochs": param_dict['epochs'], 
                "activation_func": param_dict['activation_func'], 
                "final_train_score": train_acc, 
                "valid_score": valid_acc, 
                "test_score": test_acc
    }, ignore_index=True)
    df.to_csv("./grid_search_" + filename + ".csv")
    

In [4]:
from itertools import product
import numpy as np 

def grid_search(parameters, filename):
    # Convertion from dict to product of dicts
    
    keys, values = zip(*parameters.items())
    for prod in product(*values):
        param_dict = dict(zip(keys, prod))
        print(param_dict)
        model = Net(
            dataset.num_features,
            dataset.num_classes,
            param_dict
        ).to(device)

        optim = torch.optim.Adam(
            model.parameters(), 
            lr=0.01, 
            weight_decay=0
        )
        
        avg = np.empty([1,])
        max_val = 0
        early_stop_cnt = 0
        train_acc = val_acc = test_acc = 0
        for epoch in range(param_dict['epochs']):
            # train
            model.train()
            optim.zero_grad()
            out = model(data.x, data.edge_index)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optim.step()
            
            # valid
            accs = []
            with torch.no_grad():
                model.eval()
                out = model(data.x, data.edge_index)
                for _, mask in data('train_mask', 'val_mask', 'test_mask'):
                    acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
                    accs.append(acc)
            
            
            train_acc, val_acc, test_acc = accs
            
        print(train_acc, val_acc, test_acc)
        log_stats(param_dict, train_acc, val_acc, test_acc, filename)
        
        
    



In [ ]:
## grid search for amazon photos
grid_search(hyper_parameters, "photos")

In [5]:
## load datset
dataset = 'Photo'
path = os.path.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())

## seeds
torch.manual_seed(0)

## add train / valid / test masks
dataset.shuffle()
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)

print(dataset.num_classes) # 10
# 10 * 20 --> 200
data.train_mask[:200] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[200:700] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[700:1700] = True

data.to(device)

8


Data(edge_index=[2, 238162], test_mask=[7650], train_mask=[7650], val_mask=[7650], x=[7650, 745], y=[7650])

In [6]:

grid_search(hyper_parameters, "computers")

{'depth': 1, 'heads': 1, 'epochs': 50, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.29499998688697815 0.23999999463558197 0.21199999749660492
{'depth': 1, 'heads': 1, 'epochs': 50, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.27000001072883606 0.2240000069141388 0.19499999284744263
{'depth': 1, 'heads': 1, 'epochs': 50, 'activation_func': <function tanh at 0x00000233FF95D700>}


C:\Users\kskev\miniconda3\envs\dl_project_2021\lib\site-packages\torch\nn\functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0.2849999964237213 0.23600000143051147 0.2070000022649765
{'depth': 1, 'heads': 1, 'epochs': 100, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.5249999761581421 0.46000000834465027 0.4390000104904175
{'depth': 1, 'heads': 1, 'epochs': 100, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.4650000035762787 0.44200000166893005 0.41499999165534973
{'depth': 1, 'heads': 1, 'epochs': 100, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.5699999928474426 0.4959999918937683 0.47699999809265137
{'depth': 1, 'heads': 1, 'epochs': 150, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.625 0.5379999876022339 0.5120000243186951
{'depth': 1, 'heads': 1, 'epochs': 150, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.24500000476837158 0.2980000078678131 0.289000004529953
{'depth': 1, 'heads': 1, 'epochs': 150, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.6549999713897705 0.5559999942779541 0.5419999957084656
{'depth': 1, 'heads': 1

0.7149999737739563 0.5640000104904175 0.5479999780654907
{'depth': 1, 'heads': 2, 'epochs': 450, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.9300000071525574 0.7319999933242798 0.7089999914169312
{'depth': 1, 'heads': 2, 'epochs': 500, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.9399999976158142 0.7599999904632568 0.7369999885559082
{'depth': 1, 'heads': 2, 'epochs': 500, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.8550000190734863 0.7020000219345093 0.671999990940094
{'depth': 1, 'heads': 2, 'epochs': 500, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.9399999976158142 0.7200000286102295 0.6990000009536743
{'depth': 1, 'heads': 3, 'epochs': 50, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.27000001072883606 0.2240000069141388 0.19499999284744263
{'depth': 1, 'heads': 3, 'epochs': 50, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.27000001072883606 0.2240000069141388 0.19499999284744263
{'depth': 1, '

0.4699999988079071 0.4580000042915344 0.4390000104904175
{'depth': 1, 'heads': 4, 'epochs': 350, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.574999988079071 0.46399998664855957 0.43799999356269836
{'depth': 1, 'heads': 4, 'epochs': 400, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.925000011920929 0.7099999785423279 0.6919999718666077
{'depth': 1, 'heads': 4, 'epochs': 400, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.8999999761581421 0.7260000109672546 0.7080000042915344
{'depth': 1, 'heads': 4, 'epochs': 400, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.6700000166893005 0.5059999823570251 0.5
{'depth': 1, 'heads': 4, 'epochs': 450, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.9350000023841858 0.7419999837875366 0.703000009059906
{'depth': 1, 'heads': 4, 'epochs': 450, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.9200000166893005 0.7440000176429749 0.7250000238418579
{'depth': 1, 'heads': 4, 'epoch

...

0.7049999833106995 0.6439999938011169 0.6079999804496765
{'depth': 5, 'heads': 1, 'epochs': 250, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.5550000071525574 0.45399999618530273 0.42500001192092896
{'depth': 5, 'heads': 1, 'epochs': 250, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.7649999856948853 0.699999988079071 0.675000011920929
{'depth': 5, 'heads': 1, 'epochs': 300, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.625 0.550000011920929 0.5329999923706055
{'depth': 5, 'heads': 1, 'epochs': 300, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.27000001072883606 0.2240000069141388 0.19499999284744263
{'depth': 5, 'heads': 1, 'epochs': 300, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.800000011920929 0.7879999876022339 0.7269999980926514
{'depth': 5, 'heads': 1, 'epochs': 350, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.7350000143051147 0.6800000071525574 0.6309999823570251
{'depth': 5, 'heads': 1, 'ep

0.824999988079071 0.7900000214576721 0.7680000066757202
{'depth': 5, 'heads': 3, 'epochs': 150, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.20999999344348907 0.28200000524520874 0.2639999985694885
{'depth': 5, 'heads': 3, 'epochs': 150, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.8450000286102295 0.843999981880188 0.8029999732971191
{'depth': 5, 'heads': 3, 'epochs': 200, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.8100000023841858 0.7720000147819519 0.7080000042915344
{'depth': 5, 'heads': 3, 'epochs': 200, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.4749999940395355 0.5040000081062317 0.44999998807907104
{'depth': 5, 'heads': 3, 'epochs': 200, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.8500000238418579 0.8299999833106995 0.7940000295639038
{'depth': 5, 'heads': 3, 'epochs': 250, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.8700000047683716 0.8519999980926514 0.8399999737739563
{'depth': 5, '

0.6000000238418579 0.550000011920929 0.515999972820282
{'depth': 5, 'heads': 5, 'epochs': 50, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.27000001072883606 0.2240000069141388 0.19499999284744263
{'depth': 5, 'heads': 5, 'epochs': 50, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.7350000143051147 0.7059999704360962 0.6620000004768372
{'depth': 5, 'heads': 5, 'epochs': 100, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.800000011920929 0.7839999794960022 0.7229999899864197
{'depth': 5, 'heads': 5, 'epochs': 100, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.5950000286102295 0.5540000200271606 0.515999972820282
{'depth': 5, 'heads': 5, 'epochs': 100, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.8199999928474426 0.7599999904632568 0.7540000081062317
{'depth': 5, 'heads': 5, 'epochs': 150, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.8149999976158142 0.7720000147819519 0.722000002861023
{'depth': 5, 'heads'

0.8799999952316284 0.843999981880188 0.8640000224113464
{'depth': 5, 'heads': 6, 'epochs': 450, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.6349999904632568 0.6399999856948853 0.597000002861023
{'depth': 5, 'heads': 6, 'epochs': 450, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.9100000262260437 0.878000020980835 0.8740000128746033
{'depth': 5, 'heads': 6, 'epochs': 500, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.8799999952316284 0.878000020980835 0.8550000190734863
{'depth': 5, 'heads': 6, 'epochs': 500, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.6050000190734863 0.5979999899864197 0.5609999895095825
{'depth': 5, 'heads': 6, 'epochs': 500, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.9100000262260437 0.8740000128746033 0.8610000014305115
{'depth': 5, 'heads': 7, 'epochs': 50, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.7300000190734863 0.6840000152587891 0.6520000100135803
{'depth': 5, 'heads'

0.8600000143051147 0.8479999899864197 0.8600000143051147
{'depth': 5, 'heads': 8, 'epochs': 350, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.75 0.6800000071525574 0.6430000066757202
{'depth': 5, 'heads': 8, 'epochs': 350, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.8949999809265137 0.8640000224113464 0.8659999966621399
{'depth': 5, 'heads': 8, 'epochs': 400, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.9049999713897705 0.8619999885559082 0.8650000095367432
{'depth': 5, 'heads': 8, 'epochs': 400, 'activation_func': <function relu at 0x00000233FF95BC10>}
0.7450000047683716 0.6800000071525574 0.6430000066757202
{'depth': 5, 'heads': 8, 'epochs': 400, 'activation_func': <function tanh at 0x00000233FF95D700>}
0.925000011920929 0.8840000033378601 0.8730000257492065
{'depth': 5, 'heads': 8, 'epochs': 450, 'activation_func': <function elu at 0x00000233FF95BE50>}
0.8999999761581421 0.8759999871253967 0.8709999918937683
{'depth': 5, 'heads': 8, 'epoc